In [1]:
# empty list
data_list = []

# need to ensure a clean run each time
with open('day10_test1.txt') as f:
    for line in f:
        data_list.append(line.strip('\n').split(' '))
       
       
asteroid_dict = {}


for i in range(len(data_list)):
   
    # convert to list of vals
    limit_row = list(data_list[i][0])
   
    
    for j in range(len(limit_row)):
        
        if limit_row[j] == '#':
           
            # add to asteroid dict
            asteroid_dict[(j,i)] = 'asteroid'

In [2]:
from scipy.spatial import distance


# new function: euclidean distance off coord - total laziness
def eucl_funct(coord1, coord2):
    
    return distance.euclidean(coord1, coord2)

In [3]:
def dir_x_funct(asteroid_x, val2):
    
    diff = val2 - asteroid_x
   
    if diff < 0:
       
        return "Negative"
   
    else:
        return "Positive"

def dir_y_funct(asteroid_y, val2):
    
    diff = val2 - asteroid_y # due to y_val axis being "backwards" we switch
   
    if diff < 0:
       
        return "Negative"
   
    else:
        return "Positive"
    
# equation to product slope
def slope_funct(coord1, coord2):
    
    # unpack
    x1,y1 = coord1
    x2,y2 = coord2
    
    try:
        return (y2 - y1) / (x2-x1) # slope is slightly diff.
   
    except:
        return None # need to be extreme here....sloppy sloppy
    
# x & y 
def int_funct(coord1, coord2):
    """ Calculate intercept"""
    
    # unpack
    x1,y1 = coord1
    x2,y2 = coord2
    
    if x1 == x2:
        return None # no y intercept
    
    slope = slope_funct(coord1, coord2)
    return -x1*slope+y1

# revuild y-val
def y_scale(y_asteroid, y_comp):
    """ Rescale from 0 - N to +N to -N with y_asteroid = 0"""
    
    return y_asteroid - y_comp

In [11]:
y_scale(3,4)

-1

In [12]:
slope_funct((8,0), (12,-1))

-0.25

In [39]:
coord1 = (8,3)
y_zero = coord1[1]
new_y = y_scale(y_zero, coord1[1])
print(new_y)

# dl from asteroid
try:
    del asteroid_dict[coord1]
except:
    print("Already removed")
    
coord1 = (coord1[0], new_y)

# build a few dfs to store info: 

total_vert = []
total_neg_vert = []
zone_1 = []
zone_2 = []
zone_3 = []
zone_4 = []

# count total distinct
for coord2 in asteroid_dict.keys():
     
    # update y_vals 
    new_y = y_scale(y_zero, coord2[1])
    coord2 = (coord2[0], new_y)
    
    # calculate slope & intercept:
    slope = slope_funct(coord1, coord2)
    intercept = int_funct(coord1, coord2)

    # DIRECTION OF X & Y RELATIVE TO ASTEROID
    dir_x = dir_x_funct(coord1[0], coord2[0])
    dir_y = dir_y_funct(coord1[1], coord2[1])
    
    # calculate euclidean distance
    eucl_dist = eucl_funct(coord1, coord2)
    
    # build tuple 
    coord2 = (slope, eucl_dist, coord2)

    # See if there is a starting value:
    if slope == None and dir_y == "Positive":
        total_vert.append(coord2)
        
    elif slope == None and dir_y == "Negative":
        total_neg_vert.append(coord2)        
    
    # add to list for zone 1
    elif slope >= 0 and dir_x == "Positive" and dir_y == "Positive":
        zone_1.append(coord2)
    
    # elif to list for zone 2
    elif slope <= 0 and dir_x == "Positive" and dir_y == "Negative":
        zone_2.append(coord2)

    # elif to list for zone 3
    elif slope >= 0 and dir_x == "Negative" and dir_y == "Negative":
        zone_3.append(coord2)

    # elif to list for zone 3
    elif slope <= 0 and dir_x == "Negative" and dir_y == "Positive":
        zone_4.append(coord2)
    else:
        print("Issue")

0
Already removed


In [40]:
def blast_asteroids(slope, zone_list, rank_val):

    temp_vals = []

    for lines in zone_list:

        if lines[0] == slope:
            temp_vals.append(lines)

    # sort the temp_vals
    sorted_list = sorted(temp_vals,key = lambda x: x[1])

    if len(sorted_list) > 0:
        val_remove = sorted_list[::-1].pop()
        print(rank_val, val_remove)
        rank_val += 1
        zone_list.remove(val_remove)
    
    return zone_list, rank_val

In [41]:

i = 1

while i < 36:
    
    # handling vertical
    dist_vals = sorted(set([x[0] for x in total_vert]))[::-1]
    for slope in dist_vals:
        total_vert, i = blast_asteroids(slope, total_vert, i)
    print("finished vertical loop")
    # handling zone 1
    dist_vals = sorted(set([x[0] for x in zone_1]))[::-1]
    for slope in dist_vals:
        zone_1, i = blast_asteroids(slope, zone_1, i)
    print("Finished zone 1")
    
    # handling zone 2
    dist_vals = sorted(set([x[0] for x in zone_2]))[::-1]
    for slope in dist_vals:
        zone_2, i = blast_asteroids(slope, zone_2, i)
    print("Finished zone 2")
    
    # handling negative vertical
    dist_vals = sorted(set([x[0] for x in total_neg_vert]))[::-1]
    for slope in dist_vals:
        total_neg_vert, i = blast_asteroids(slope, total_neg_vert, i)

    print("finished negative vert")
    
    # handling zone 3
    dist_vals = sorted(set([x[0] for x in zone_3]))[::-1]
    for slope in dist_vals:
        zone_3, i = blast_asteroids(slope, zone_3, i)
    print("Finished zone 3")
    
    # handling zone 4
    dist_vals = sorted(set([x[0] for x in zone_4]))[::-1]
    for slope in dist_vals:
        zone_4, i = blast_asteroids(slope, zone_4, i)
    print("Finished Zone 4")

1 (None, 2.0, (8, 2))
finished vertical loop
2 (3.0, 3.1622776601683795, (9, 3))
3 (2.0, 2.23606797749979, (9, 2))
4 (1.5, 3.605551275463989, (10, 3))
5 (1.0, 1.4142135623730951, (9, 1))
6 (0.6666666666666666, 3.605551275463989, (11, 2))
7 (0.5, 4.47213595499958, (12, 2))
8 (0.3333333333333333, 3.1622776601683795, (11, 1))
9 (0.2857142857142857, 7.280109889280518, (15, 2))
10 (0.25, 4.123105625617661, (12, 1))
11 (0.2, 5.0990195135927845, (13, 1))
12 (0.16666666666666666, 6.082762530298219, (14, 1))
13 (0.14285714285714285, 7.0710678118654755, (15, 1))
14 (0.0, 4.0, (12, 0))
Finished zone 1
15 (-0.125, 8.06225774829855, (16, -1))
16 (-0.14285714285714285, 7.0710678118654755, (15, -1))
17 (-0.5, 2.23606797749979, (10, -1))
Finished zone 2
finished negative vert
18 (0.25, 4.123105625617661, (4, -1))
19 (0.16666666666666666, 6.082762530298219, (2, -1))
Finished zone 3
20 (-0.0, 6.0, (2, 0))
21 (-0.125, 8.06225774829855, (0, 1))
22 (-0.14285714285714285, 7.0710678118654755, (1, 1))
23 (-0.

In [ ]:
zone_1

In [ ]:
for slope in dist_vals:
    
    temp_vals = []
    
    for lines in zone_1:
        
        if lines[0] == slope:
            temp_vals.append(lines)
    
    # sort the temp_vals
    sorted_list = sorted(temp_vals,key = lambda x: x[1])
    if len(sorted_list) > 0:
        val_remove = sorted_list[::-1].pop()
        print(val_remove[2])
        zone_1.remove(val_remove)

In [ ]:
sorted(zone_1, key = lambda x: (x[0], x[1]))

In [270]:
zone_2

[(10, -1), (15, -1), (16, -1)]

In [ ]:
##### Run process: 

while i < 10: 